# Initial definitions

In [1]:
%env HADOOP_VERSION     2.9.2
%env HADOOP_PATH hadoop-2.9.2

env: HADOOP_VERSION=2.9.2
env: HADOOP_PATH=hadoop-2.9.2


# Preparing the environment

## Downloading Hadoop

In [ ]:
!wget http://ftp.unicamp.br/pub/apache/hadoop/common/hadoop-${HADOOP_VERSION}/hadoop-${HADOOP_VERSION}.tar.gz -q --show-progress

## Extracting compressed files and removing .tar

In [2]:
!tar -xvf hadoop-${HADOOP_VERSION}.tar.gz >/dev/null 
# !rm       hadoop-${HADOOP_VERSION}.tar.gz

## Discovering the Java path

In [3]:
!dirname $(dirname $(readlink -f $(which javac)))

/usr/lib/jvm/java-8-openjdk-amd64


## Setting the Java path envvar

We also added it to user's .bashrc so it will be loaded as the nodes perform ssh connections.

In [4]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64

env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [5]:
!echo "export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64 " > .bashrc

# Hadoop in Standalone Mode (local)

## MapReduce in the local filesystem - word count example

In [6]:
!${HADOOP_PATH}/bin/hadoop jar ${HADOOP_PATH}/share/hadoop/mapreduce/hadoop-mapreduce-examples-${HADOOP_VERSION}.jar  \
                               wordcount \
                               ./resources/examples/newyorknewyork.txt \
                               ./output

19/07/07 05:14:11 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
19/07/07 05:14:11 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
19/07/07 05:14:11 INFO input.FileInputFormat: Total input files to process : 1
19/07/07 05:14:11 INFO mapreduce.JobSubmitter: number of splits:1
19/07/07 05:14:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1543156223_0001
19/07/07 05:14:11 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
19/07/07 05:14:11 INFO mapreduce.Job: Running job: job_local1543156223_0001
19/07/07 05:14:11 INFO mapred.LocalJobRunner: OutputCommitter set in config null
19/07/07 05:14:11 INFO output.FileOutputCommitter: File Output Committer Algorithm version is 1
19/07/07 05:14:11 INFO output.FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
19/07/07 05:14:11 INFO mapred.LocalJ

### Listing files in the output folder

In [7]:
!ls ./output/

part-r-00000  _SUCCESS


### Reading output file

In [8]:
! cat ./output/part-r-00000

And	3
Come	1
Head	1
I	8
I'll	1
I'm	3
If	2
In	1
It's	1
King	1
My	1
New	13
Right	2
Start	1
That	2
These	2
They	3
Top	2
York	13
You	1
a	3
about	2
all	1
am	2
anywhere	2
are	3
away	2
baby	1
be	1
bet	1
blues	2
brand	2
can	2
city	2
come	1
doesn't	1
find	2
gonna	2
have	1
heap	2
heart	1
hill	3
in	3
it	8
just	1
king	2
know	1
leaving	1
list	1
little	2
longing	1
make	6
melted	1
melting	1
never	1
new	2
news	1
of	10
old	2
on	1
part	1
shoes	1
sleep	1
sleeps	1
spreading	1
start	2
stray	1
that	2
the	8
there	3
through	2
to	6
today	1
town	2
up	3
vagabond	1
very	1
wake	2
want	3
you	2


# Hadoop in Pseudo-Distributed Mode

## Preparing the environment

### Starting sshd server

Check `/binder/postBuild` and `/resources/configs/ssh/sshd_config` files for more details

In [9]:
!/usr/sbin/sshd -f resources/configs/ssh/sshd_config 

### Adding names to know hosts 

Commands below stablish ssh connections to used host names/ips. This step avoids yes/no host confirmation.

In [10]:
!ssh -o "StrictHostKeyChecking no" $USER@localhost -p 8822 -C "exit" 
!ssh -o "StrictHostKeyChecking no" $USER@127.0.0.1 -p 8822 -C "exit"
!ssh -o "StrictHostKeyChecking no" $USER@0.0.0.0   -p 8822 -C "exit"

### Adding ssh options to Hadoop via envvar

* connecting in a diferent port (`-p 8822`)
* avoiding host key checking (`-o StrictHostKeyChecking=no`)

In [11]:
%env HADOOP_SSH_OPTS= -o StrictHostKeyChecking=no -p 8822

env: HADOOP_SSH_OPTS=-o StrictHostKeyChecking=no -p 8822


### Copying configurations files to Hadoop folder

Check the configuration files accordingly to the Hadoop version. 
Refer to the `/resources/configs/hadoop/<version>`.

In [12]:
!cp resources/configs/hadoop/${HADOOP_VERSION}/core-site.xml   ${HADOOP_PATH}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/hdfs-site.xml   ${HADOOP_PATH}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/mapred-site.xml ${HADOOP_PATH}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/yarn-site.xml   ${HADOOP_PATH}/etc/hadoop/

## Formatting the filesystem

In [13]:
!${HADOOP_PATH}/bin/hdfs namenode -format -force -nonInteractive

19/07/07 05:14:43 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = 47958a0ef968/172.17.0.2
STARTUP_MSG:   args = [-format, -force, -nonInteractive]
STARTUP_MSG:   version = 2.9.2
STARTUP_MSG:   classpath = /home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/etc/hadoop:/home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/share/hadoop/common/lib/xmlenc-0.52.jar:/home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/share/hadoop/common/lib/nimbus-jose-jwt-4.41.1.jar:/home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/share/hadoop/common/lib/httpcore-4.4.4.jar:/home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/share/hadoop/common/lib/junit-4.11.jar:/home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/share/hadoop/common/lib/activation-1.1.jar:/home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/s

## Starting DFS (NameNode, SecondaryNameNode, and DataNode daemons)

In [15]:
!${HADOOP_PATH}/sbin/start-dfs.sh
!jps

Starting namenodes on [localhost]
localhost: starting namenode, logging to /home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/logs/hadoop-matheus-namenode-47958a0ef968.out
localhost: starting datanode, logging to /home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/logs/hadoop-matheus-datanode-47958a0ef968.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /home/matheus/local-home/git/github/binderhub-hadoop/hadoop-2.9.2/logs/hadoop-matheus-secondarynamenode-47958a0ef968.out
743 DataNode
617 NameNode
1035 Jps
908 SecondaryNameNode


## MapReduce - Word count example 

### Creating folders in the distributed file system

In [16]:
!${HADOOP_PATH}/bin/hdfs dfs -mkdir /user/
!${HADOOP_PATH}/bin/hdfs dfs -mkdir /user/matheus/
!${HADOOP_PATH}/bin/hdfs dfs -mkdir /user/matheus/input/

### Copying a file to a folder in the distributed file system

In [17]:
!${HADOOP_PATH}/bin/hdfs dfs -put ./resources/examples/newyorknewyork.txt /user/matheus/input/

### Listing files in a folder of the distributed file system

In [18]:
!${HADOOP_PATH}/bin/hdfs dfs -ls /user/matheus/input/

Found 1 items
-rw-r--r--   1 matheus supergroup        865 2019-07-07 05:16 /user/matheus/input/newyorknewyork.txt


### Retrieving the contents of a file in the distributed file system

In [19]:
!${HADOOP_PATH}/bin/hdfs dfs -cat /user/matheus/input/newyorknewyork.txt

Start spreading the news
I am leaving today
I want to be a part of it
New York New York
These vagabond shoes
They are longing to stray
Right through the very heart of it
New York New York
I want to wake up in that city
That doesn't sleep
And find I'm king of the hill
Top of the heap
My little town blues
They are melting away
I gonna make a brand new start of it
In old New York
If I can make it there
I'll make it anywhere
It's up to you
New York New York
New York New York
I want to wake up in that city
That never sleeps
And find I'm king of the hill
Top of the list
Head of the heap
King of the hill
These are little town blues
They have all melted away
I am about to make a brand new start of it
Right there in old New York
And you bet baby
If I can make it there
You know I'm gonna make it just about anywhere
Come on come through
New York New York New York


### Running MapReduce job in Pseudo-Distributed Mode

In [20]:
!${HADOOP_PATH}/bin/hadoop jar ./${HADOOP_PATH}/share/hadoop/mapreduce/hadoop-mapreduce-examples-${HADOOP_VERSION}.jar  \
                                 wordcount \
                                 /user/matheus/input /user/matheus/output

19/07/07 05:16:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/07/07 05:16:29 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 0 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
19/07/07 05:16:30 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 1 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
19/07/07 05:16:31 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 2 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
19/07/07 05:16:32 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 3 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
19/07/07 05:16:33 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Alrea

### Listing files in the output folder

In [21]:
!./${HADOOP_PATH}/bin/hdfs dfs -ls /user/matheus/output/

ls: `/user/matheus/output/': No such file or directory


### Reading output file

In [22]:
!./${HADOOP_PATH}/bin/hdfs dfs -cat /user/matheus/output/part-r-00000

cat: `/user/matheus/output/part-r-00000': No such file or directory


# Hadoop YARN in Pseudo-Distributed Mode

## Preparing the environment

### Copying configurations files to Hadoop folder

In [ ]:
!cp resources/configs/hadoop/${HADOOP_VERSION}/mapred-site.xml ${HADOOP_PATH}/etc/hadoop/
!cp resources/configs/hadoop/${HADOOP_VERSION}/yarn-site.xml   ${HADOOP_PATH}/etc/hadoop/

## Starting YARN

In [ ]:
!${HADOOP_PATH}/sbin/start-dfs.sh
!${HADOOP_PATH}/sbin/start-yarn.sh
!jps

## Running MapReduce job via YARN

In [ ]:
!${HADOOP_PATH}/bin/yarn jar ./${HADOOP_PATH}/share/hadoop/mapreduce/hadoop-mapreduce-examples-${HADOOP_VERSION}.jar  \
                                 wordcount \
                                 /user/matheus/input /user/matheus/output

### Listing files in the output folder

In [ ]:
!./${HADOOP_PATH}/bin/hdfs dfs -ls /user/matheus/output/

### Reading output file

In [ ]:
!./${HADOOP_PATH}/bin/hdfs dfs -cat /user/matheus/output/part-r-00000

In [ ]:
#!${HADOOP_PATH}/sbin/stop-dfs.sh